In [ ]:
username = ""
source = ""

In [ ]:
import NBInclude: @nbinclude
@nbinclude("RecommendationsBase.ipynb");

## Curated Feed

In [ ]:
curated_feed(200)

## Continue Watching

In [ ]:
continue_watching_tv(5)

In [ ]:
continue_watching_specials(5)

## Plan to Watch

In [ ]:
plan_to_watch(10)